<a href="https://colab.research.google.com/github/bala-codes/Natural-Language-Processing-NLP/blob/master/Neural%20Machine%20Translation/1.%20Seq2Seq%20%5BEnc%20%2B%20Dec%5D%20Model%20for%20Neural%20Machine%20Translation%20(Without%20Attention%20Mechanism).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Comprehensive Guide to Neural Machine Translation using Seq2Seq Modelling using PyTorch

## In this post, we will be building a sequence to sequence deep learning model using PyTorch and TorchText. Here I am doing an German to English neural machine translation. But the same concept can be extended to other problems such as Named Entity Recognition (NER), Text Summarization etc,.

# Table of Contents:
## 1. Introduction
## 2. Data Preparation and Pre-processing
## 3. Long Short Term Memory (LSTM) - Under the Hood
## 4. Encoder Model Architecture (Seq2Seq)¶
## 5. Encoder Code Implementation (Seq2Seq)
## 6. Decoder Model Architecture (Seq2Seq)
## 7. Decoder Code Implementation (Seq2Seq)
## 8. Seq2Seq (Encoder + Decoder) Interface
## 9. Seq2Seq (Encoder + Decoder) Code Implementation
## 10. Seq2Seq Model Training
## 11. Seq2Seq Model Inference

# 1. Introduction

Here I am doing a German to English neural machine translation. But the same concept can be extended to other problems such as Named Entity Recognition (NER), Text Summarization, etc,.

So the Sequence to Sequence (seq2seq) model in this post uses an encoder-decoder architecture, which uses a type of RNN called LSTM (Long Short Term Memory), where the encoder neural network encodes the input german sequence into a single vector, also called as a Context Vector.
This Context Vector is said to contain the abstract representation of the input german sequence.

This vector is then passed into the decoder neural network, which is used to output the corresponding English translation sentence, one word at a time.

# Necessary Imports 

In [ ]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
'''
# Seeding for reproducible results everytime
SEED = 777

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True'''

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 1.1MB 14.5MB/s 


'\n# Seeding for reproducible results everytime\nSEED = 777\n\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.deterministic = True'

# 2. Data Preparation & Pre-processing

Loading the SpaCy's vocabulary for our desired languages. SpaCy also supports many languages like french, german etc,.



In [ ]:
!python -m spacy download en --quiet
!python -m spacy download de --quiet

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 531kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
spacy_german = spacy.load("de")
spacy_english = spacy.load("en")

Now let's create custom tokenization methods for the languages. Tokenization is a process of breaking the sentence into a list of individual tokens (words).

We can make use of PyTorch's TorchText library for data pre-processing and SpaCy for vocabulary building (English and German) & tokenization of our data.

In [ ]:
def tokenize_german(text):
  return [token.text for token in spacy_german.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

### Sample Run ###

sample_text = "I love machine learning"
print(tokenize_english(sample_text))

['I', 'love', 'machine', 'learning']


Torch text is a powerful library for making the text data ready for variety of NLP tasks. It has all the tools to perform preprocessing on the textual data.

Let's see some of the process it can do,

1. Train/ Valid/ Test Split: partition your data into a specified train/ valid/ test set.

2. File Loading: load the text corpus of various formats (.txt,.json,.csv).
3. Tokenization: breaking sentences into list of words.
4. Vocab: Generate a list of vocabulary from the text corpus.
5. Words to Integer Mapper: Map words into integer numbers for the entire corpus and vice versa.
6. Word Vector: Convert a word from higher dimension to lower dimension (Word Embedding).
7. Batching: Generate batches of sample.

So once we get to understand what can be done in torch text, let's talk about how it can be implemented in the torch text module. Here we are going to make use of 3 classes under torch text.

1. Fields :
> This is a class under the torch text, where we specify how the preprocessing should be done on our data corpus.
2. TabularDataset : 
> Using this class, we can actually define the Dataset of columns stored in CSV, TSV, or JSON format and also map them into integers.
3. BucketIterator :
> Using this class, we can perform padding our data for approximation and make batches with our data for model training.

Here our source language (SRC - Input) is German and target language (TRG - Output) is English. We also add 2 extra tokens "start of sequence" <sos> and "end of sequence" <EOS> for effective model training.

In [ ]:
german = Field(tokenize=tokenize_german,
               lower=True,
               init_token="<sos>",
               eos_token="<eos>")

english = Field(tokenize=tokenize_english,
               lower=True,
               init_token="<sos>",
               eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".de", ".en"),
                                                    fields=(german, english))

german.build_vocab(train_data, max_size=10000, min_freq=3)
english.build_vocab(train_data, max_size=10000, min_freq=3)

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 671kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 175kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 167kB/s]


In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")

Unique tokens in source (de) vocabulary: 5376
Unique tokens in target (en) vocabulary: 4556


In [3]:
# dir(english.vocab)

print(english.vocab.__dict__.keys())
print(list(english.vocab.__dict__.values()))
e = list(english.vocab.__dict__.values())
for i in e:
  print(i)

'\nprint(english.vocab.__dict__.keys())\nprint(list(english.vocab.__dict__.values()))\ne = list(english.vocab.__dict__.values())\nfor i in e:\n  print(i)'

In [ ]:
word_2_idx = dict(e[3])
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k

# Dataset sneek peek

In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.'], ['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.']])


After setting the language pre-processing criteria, the next step is to create batches of training, testing and validation data using iterators.

Creating batches is an exhaustive process, luckily we can make use of TorchText's iterator libraries.

Here we are using BucketIterator for effective padding of source and target sentences. We can access the source (german) batch of data using .src attribute and it's correspondign (english) batch of data using .trg attribute.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

## Actual text data before tokenized

In [ ]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
  max_len_ger.append(len(data.src))
  max_len_eng.append(len(data.trg))
  if count < 10 :
    print("German - ",*data.src, " Length - ", len(data.src))
    print("English - ",*data.trg, " Length - ", len(data.trg))
    print()
  count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

German -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13
English -  two young , white males are outside near many bushes .  Length -  11

German -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8
English -  several men in hard hats are operating a giant pulley system .  Length -  12

German -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10
English -  a little girl climbing into a wooden playhouse .  Length -  9

German -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15
English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15

German -  zwei männer stehen am herd und bereiten essen zu .  Length -  10
English -  two men are at the stove preparing food .  Length -  9

German -  ein mann in grün hält eine gitarre , während der andere mann sein hemd ansieht .  Length -  16
English -  a man in green holds a guitar while the other

In [ ]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.src.shape, data.trg.shape)
    print()
    print("German - ",*data.src, " Length - ", len(data.src))
    print()
    print("English - ",*data.trg, " Length - ", len(data.trg))
    temp_ger = data.src
    temp_eng = data.trg
    count += 1

Shapes torch.Size([23, 32]) torch.Size([32, 32])

German -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([  5,   5,   5,   8,   8,   5,   5,   8,   5,   5, 105,  15,   8,   8,
          8,   5,   5,  18,   5,   5,   5, 105,  18,   5,   8,   5,   5,   5,
         18,   5,   8,   5], device='cuda:0') tensor([   0, 1380, 3448,  165,  330,   13,   26,   16,   70,   13,   30, 2239,
         230,   36,   16, 1227,   96,   65,   13,    0,   13,   54,   30,   13,
          16,   13,    7,   70,   30,   49,  926,   96], device='cuda:0') tensor([ 572,  533, 1575,    7,   16,   11,    7,   11,   26,    7,    9,   11,
           9,   22,    7,   29,   13,   57,   10,    9,    7,   17,  185,    7,
           7,  412, 1032,   26,  179,    9,  327,   13], device='cuda:0') tensor([  11,   11,   49,  234,   10,   14, 2485,   14,   11,  219,   18,   15,
          17,   41,    6,   12,    0,   55,    5,   15,    6

In [ ]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

I just experimented with a batch size of 32 and a sample target batch is shown below. The sentences are tokenized into list of words and indexed according to the vocabulary. The "pad" token gets an index of 1.

Each column corresponds to a sentence indexed into numbers and we have 32 such sentences in a single target batch and the number of rows corresponds to the maximum length of that sentence. Short sentences are padded with 1 to compensate.
The table (Idx.csv) contains the numerical indices of the words, which is later fed into the word embedding and converted into dense representation for Seq2Seq processing.

In [ ]:
df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,4,4,4,4,21,4,4,4,4,4,110,7,4,4,4,4,4,16,4,4,4,110,16,4,4,4,7,4,16,55,21,4
3,4314,178,1585,120,387,174,34,14,24,9,30,939,200,38,14,1421,24,63,9,523,9,19,30,9,14,168,34,53,30,22,106,24
4,665,42,55,6,14,125,6,13,34,6,17,8,12,12,6,10,9,37,11,15,6,22,17,6,73,42,6,34,17,1553,33,9
5,13,210,91,29,11,13,731,4,13,148,37,7,115,19,4,36,1708,20,4,22,4,25,633,7,6,433,132,13,130,102,10,6
6,27,524,56,15,4,371,846,26,122,11,123,1057,30,17,31,8,51,4,35,4,52,17,2395,29,4,9,10,4,6,67,351,4
7,55,13,60,165,24,6,380,81,42,4,15,15,8,359,23,74,4,88,8,59,23,8,4,23,26,22,165,194,4,91,4,26
8,11,4,4,4,161,27,4,11,97,2314,16,448,149,129,11,392,0,15,4,42,10,4,14,10,109,4,426,23,284,8,518,23
9,35,1283,598,700,55,139,0,26,11,23,6,0,133,11,204,49,922,46,118,3311,151,70,57,37,11,29,12,41,370,185,12,11


In [ ]:
df_eng_word = pd.DataFrame(columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_word = df_eng_idx.replace(idx_2_word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,a,a,a,a,an,a,a,a,a,a,four,the,a,a,a,a,a,two,a,a,a,four,two,a,a,a,the,a,two,child,an,a
3,proud,dark,redheaded,lady,adult,guy,boy,woman,young,man,men,pitcher,band,group,woman,fisherman,young,children,man,vendor,man,people,men,man,woman,middle,boy,little,men,wearing,asian,young
4,father,-,child,in,woman,walks,in,with,boy,in,are,on,of,of,in,is,man,playing,and,",",in,wearing,are,in,dressed,-,in,boy,are,fireman,girl,man
5,with,haired,sits,blue,and,with,swim,a,with,shorts,playing,the,older,people,a,standing,flips,at,a,wearing,a,white,speaking,the,in,aged,gray,with,working,'s,is,in
6,his,gentleman,looking,",",a,fire,trunks,black,blond,and,soccer,mound,men,are,red,on,up,a,dog,a,green,are,too,blue,a,man,is,a,in,hat,drinking,a
7,child,with,through,taking,young,in,does,jacket,-,a,",",",",on,gathered,shirt,some,a,beach,on,large,shirt,on,a,shirt,black,wearing,taking,striped,a,sits,a,black
8,and,a,a,a,male,his,a,and,hair,hawaiian,two,number,stage,together,and,rocks,<unk>,",",a,-,is,a,woman,is,top,a,pictures,shirt,kitchen,on,cup,shirt
9,dog,mustache,railing,close,child,hand,<unk>,black,and,shirt,in,<unk>,smiling,and,purple,by,',one,park,brimmed,carrying,small,outside,playing,and,blue,of,walking,preparing,ground,of,and


# 3. Long Short Term Memory (LSTM) - Under the Hood

<img src="https://cdn-images-1.medium.com/max/2560/1*sQBwBtwCwqqXY0k5O0ZvMg.png">

## The above picture shows the units present under a single LSTM Cell. I will add some references to learn more about LSTM in the last and why it works well for long sequences.

## But to simply put, Vanilla RNN, Gated Recurrent Unit (GRU) is not able to capture the long term dependencies due to its nature of design and suffers heavily by the Vanishing Gradient problem, which makes the rate of change in weights and bias values negligible, resulting in poor generalization.

## But LSTM has some special units called gates (Remember gate, Forget gate, Update gate), which helps to overcome the problems stated before.

## Inside the LSTM cell, we have a bunch of mini neural networks with sigmoid and TanH activations at the final layer and few vector adder, Concat, multiplications operations.

1. Sigmoid NN → Squishes the values between 0 and 1. Say a value closer to 0 means to forget and a value closer to 1 means to remember.

2. Embedding NN → Converts the input word indices into word embedding.

3. TanH NN → Squishes the values between -1 and 1. Helps to regulate the vector values from either getting exploded to the maximum or shrank to the minimum.

4. The hidden state and the cell state are referred to here as the context vector, which are the outputs from the LSTM cell. The input is the sentence's numerical indexes fed into the embedding NN.


# 4. Encoder Model Architecture (Seq2Seq)

## Before moving to seq2seq model, we need to create Encoder ,Decoder and create a interface between them in the seq2seq model.

## Let's pass the german input sequence "Ich liebe tief lernen" which translates to "I love deep learning" in english.




<img src="https://cdn-images-1.medium.com/max/1200/1*aNcybCTdPlrXsCwIo1OfTg.png">

## For a lighter note, let's explain the process happening in the above image. The Encoder of the Seq2Seq model takes one input at a time. Our input German word sequence is "ich Liebe Tief Lernen".

## Also, we append the start of sequence "SOS" and the end of sentence "EOS" tokens in the starting and in the ending of the input sentence.

## Therefore at 
## At time step-0, the "SOS" token is sent,
## At time step-1 the token "ich" is sent,
## At time step-2 the token "Liebe" is sent,
## At time step-3 the token "Tief" is sent,
## At time step-4 the token "Lernen" is sent,
## At time step-4 the token "EOS" is sent.

## And the first block in the Encoder architecture is the word embedding layer [shown in green block], which converts the input indexed word into a dense vector representation called word embedding (sizes - 100/200/300).

## Then our word embedding vector is sent to the LSTM cell, where it is combined with the hidden state (hs), and the cell state (cs) of the previous time step and the encoder block outputs a new hs and a cs which is passed to the next LSTM  cell. 

## It is understood that the hs and cs captured some vector representation of the sentence so far.

## At time step-0, the hidden state and cell state are either initialized fully of zeros or random numbers.

## Then after we sent pass all our input german word sequence, a context vector [shown in yellow block] (hs, cs) is finally obtained, which is a dense representation of the word sequence and can be sent to the decoder's first LSTM (hs, cs) for the corresponding English translation.

## In the above figure, we use 2 layer LSTM  architecture, where we connect the first LSTM to the second LSTM and we then we obtain 2 context vectors stacked on top as the final output.

## It is a must that we design identical encoder and decoder blocks in the seq2seq model.

## The above visualization is applicable for a single sentence from a batch. Say we have a batch size of 5 (Experimental), then we pass 5 sentences at a time to the Encoder, which looks like the below figure.

<img src="https://cdn-images-1.medium.com/max/1200/1*xP8MgIfKwjStFDUo0_W3QA.png">

##  The same concept is extended to a batch size of 5 (experimental), where we consider 5 input sentences and the first token from each sentences is sent to the encoder at a time. Each sequences in the batch is maintained to have the same length using the padding token.

# 5. Encoder Code Implementation (Seq2Seq)

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(german.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(5376, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


# 6. Decoder Model Architecture (Seq2Seq)

<img src="https://cdn-images-1.medium.com/max/800/1*FtDDCniBMb8HXYEM6PRohQ.png">

## The decoder also does a single step at a time.

## The Context Vector from the Encoder block is provided as the hidden state (hs) and cell state (cs) for the decoder's first LSTM block.

## The start of the sentence "SOS"  token is passed to the embedding NN, then passed to the first LSTM cell of the decoder, and finally, it is passed through a linear layer [Shown in Pink color], which provides an output English token prediction probabilities (4556 Probabilities), hidden state (hs), Cell State (cs). 

## The output word with the highest probability is chosen, hidden state (hs), Cell State (cs) is passed as the inputs to the next LSTM cell and this process is executed until it reaches the end of sentences "EOS".

## The subsequent layers will use the hidden and cell state from the previous time steps.

## The above visualization is applicable for a single sentence from a batch. Say we have a batch size of 5 (Experimental), then we pass 5 sentences at a time to the Encoder, which provide 5 sets of Context Vectors, and they all are passed into the Decoder, which looks like the below figure.

<img src="https://cdn-images-1.medium.com/max/2560/1*UPyGSZSuIQ52IjyFdPpm6A.png">

# The same concept is extended to a batch size of 5 (experimental), where we consider 5 input encoder's context vectors and the first token <"sos"> is sent to the decoder at a time. 

# 7. Decoder Code Implementation (Seq2Seq)

In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4556, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=4556, bias=True)
)


# 8. Seq2Seq (Encoder + Decoder) Interface

<img src="https://cdn-images-1.medium.com/max/1200/1*d9kP4XoWGnIcmyhX-g4Xvw.png">

## The final seq2seq implementation looks like the figure above.

## 1. Provide both input (German) and output (English) sentences.
## 2. Pass the input sequence to the encoder and extract context vectors.
## 3. Pass the output sequence to the decoder, context vecotr from encoder to produce the predicted output sequence.


<img src="https://cdn-images-1.medium.com/max/1200/1*7SVU_REkIUALmegTbFI9Fw.png">

In [ ]:
for batch in train_iterator:
  print(batch.src.shape)
  print(batch.trg.shape)
  break

x = batch.trg[1]
print(x)

torch.Size([21, 32])
torch.Size([25, 32])
tensor([  16,    4,  500,   21,   70,    4,  176,    4,    4,    4,    9,    4,
         209,  691,    4,    4,    4,    9, 2208,    4,    7,    4,    4,    4,
         209,   16,    4,    4,  417,   48,  113,    4], device='cuda:0')


# 9. Seq2Seq (Encoder + Decoder) Code Implementation

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs


In [ ]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5376, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4556, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=4556, bias=True)
  )
)

In [ ]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_ger = spacy.load("de")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, '/content/checkpoint-NMT')
    torch.save(model.state_dict(),'/content/checkpoint-NMT-SD')

# 10. Seq2Seq Model Training

In [ ]:
|epoch_loss = 0.0
num_epochs = 100
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))

score = bleu(test_data[1:100], model, german, english, device)
print(f"Bleu score {score*100:.2f}")

Epoch - 1 / 100
Translated example sentence 1: 
 ['yo', 'yo', 'talkie', 'warm', 'warm', 'sunhats', 'missing', 'location', 'barbecuing', 'radio', 'wilderness', 'kayaking', 'uniform', 'wii', 'squatting', 'yo', 'yo', 'talkie', '4', '4', '4', '4', 'backward', 'backward', '4', 'landscape', 'buffet', 'fast', 'sunbathers', 'shoveling', 'shoveling', 'tugging', 'tugging', 'tugging', 'flags', 'sacks', 'sacks', 'sacks', 'wooded', 'horses', 'horses', 'tends', 'tends', 'tends', 'wade', 'wade', 'drill', 'drill', 'softball', 'drill']
saving

Epoch_Loss - 3.9891204833984375

Epoch - 2 / 100
Translated example sentence 1: 
 ['a', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'a', 'a', 'a', 'a', 'a', '.', '<eos>']
Epoch_Loss - 3.4609951972961426

Epoch - 3 / 100
Translated example sentence 1: 
 ['a', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'standing', 'on', 'a', 'a', 'a', 'a', '.', '<eos>']
Epoch_Loss - 3.449892044067383

Epoch - 4 / 100
Translated example sentence 1: 
 ['a', 'man', 'in', 'a', 'blue', 'shir

In [1]:
#%load_ext tensorboard
%tensorboard --logdir runs/

UsageError: Line magic function `%tensorboard` not found.


# 11. Seq2Seq Model Inference

In [ ]:
progress  = []
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
for i, sen in enumerate(ts1):
  progress.append(TreebankWordDetokenizer().detokenize(sen))
print(progress)

In [ ]:
progress_df = pd.DataFrame(data = progress, columns=['Predicted Sentence'])
progress_df.index.name = "Epochs"
progress_df.to_csv('/content/predicted_sentence.csv')
progress_df.head()

# Model Inference

In [ ]:
model.eval()
test_sentences  = ["Zwei Männer gehen die Straße entlang", "Kinder spielen im Park.", "Diese Stadt verdient eine bessere Klasse von Verbrechern. Der Spaßvogel"]
actual_sentences  = ["Two men are walking down the street", "Children play in the park", "This city deserves a better class of criminals. The joker"]
pred_sentences = []

for idx, i in enumerate(test_sentences):
  model.eval()
  translated_sentence = translate_sentence(model, i, german, english, device, max_length=50)
  progress.append(TreebankWordDetokenizer().detokenize(translated_sentence))
  print("German : {}".format(i))
  print("Actual Sentence in English : {}".format(actual_sentences[idx]))
  print("Predicted Sentence in English : {}".format(progress[-1]))
  print()
